In [44]:
import pandas as pd
from xgboost import XGBClassifier, XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('../data/processed_data/feature_engineered.csv')
df

,customer_id,recency,frequency,price_sum,price_mean,spend_90_total,spend_90_flag
0,1,-455,1,11.77,11.770000,0.00,0.0
1,2,-444,2,89.00,44.500000,0.00,0.0
2,3,-127,5,139.47,27.894000,16.99,1.0
3,4,-110,4,100.50,25.125000,0.00,0.0
4,5,-88,11,385.61,35.055455,0.00,0.0
...,...,...,...,...,...,...,...
23565,23566,-372,1,36.00,36.000000,0.00,0.0
23566,23567,-372,1,20.97,20.970000,0.00,0.0
23567,23568,-344,3,121.70,40.566667,0.00,0.0
23568,23569,-372,1,25.74,25.740000,0.00,0.0


In [45]:
X = df[['recency', 'frequency', 'price_sum', 'price_mean']]
y = df['spend_90_total']

In [46]:
xgb_reg_spec = XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_reg_spec

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

In [47]:
xgb_reg_model = GridSearchCV(
  estimator=xgb_reg_spec,
  param_grid=dict(
    learning_rate = (0.01, 0.1, 0.2, 0.3, 0.5)
  ),
  scoring = 'neg_mean_squared_error',
  refit=True,
  cv=5
)

In [48]:
xgb_reg_model.fit(X, y)
xgb_reg_model.best_estimator_

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

In [49]:
xgb_reg_model.best_score_

-1175.975390210238

In [50]:
xgb_reg_model.best_params_

{'learning_rate': 0.1}

In [51]:
predictions_req = xgb_reg_model.predict(X)

In [52]:
predictions_req

array([ 0.9252182,  2.54172  , 12.041704 , ...,  4.643804 ,  1.1512123,
        3.1477506], dtype=float32)